<a href="https://colab.research.google.com/github/lakshyabhatnagar/Amazon-Product-Review-Sentiment-Analysis/blob/main/Amazon_Reviews_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [34]:
from google.colab import files
uploaded = files.upload()


Saving amazon_reviews.csv to amazon_reviews (2).csv


In [35]:
import pandas as pd
df=pd.read_csv('amazon_reviews.csv')


In [6]:
df.head()

,Unnamed: 0,reviewerName,overall,reviewText,reviewTime,day_diff,helpful_yes,helpful_no,total_vote,score_pos_neg_diff,score_average_rating,wilson_lower_bound
0,0,NaN,4.0,No issues.,2014-07-23,138,0,0,0,0,0.0,0.0
1,1,0mie,5.0,"Purchased this for my device, it worked as adv...",2013-10-25,409,0,0,0,0,0.0,0.0
2,2,1K3,4.0,it works as expected. I should have sprung for...,2012-12-23,715,0,0,0,0,0.0,0.0
3,3,1m2,5.0,This think has worked out great.Had a diff. br...,2013-11-21,382,0,0,0,0,0.0,0.0
4,4,2&amp;1/2Men,5.0,"Bought it with Retail Packaging, arrived legit...",2013-07-13,513,0,0,0,0,0.0,0.0


In [36]:
df=df[['overall','reviewText']]

In [8]:
df.head()

,overall,reviewText
0,4.0,No issues.
1,5.0,"Purchased this for my device, it worked as adv..."
2,4.0,it works as expected. I should have sprung for...
3,5.0,This think has worked out great.Had a diff. br...
4,5.0,"Bought it with Retail Packaging, arrived legit..."


In [9]:
df.isnull().sum()

,0
overall,0
reviewText,1


In [37]:
df.dropna(inplace=True)

In [11]:
df.isnull().sum()
df.shape

(4914, 2)

In [12]:
df['overall'].unique()

array([4., 5., 3., 1., 2.])

In [13]:
df['overall'].value_counts()

,count
overall,
5.0,3921
4.0,527
1.0,244
3.0,142
2.0,80


Since there are so many ratings with 5 stars, we will consider only the review with 5 stars as positive review, else it is a negative review.


In [38]:
df['overall']=df['overall'].apply(lambda x: 1 if x==5 else 0)

In [15]:
df['overall'].value_counts()

,count
overall,
1,3921
0,993


Now we will preprocess the data. Specifically:

*   Lowercase the sentences
*   Remove special characters
-   Remove Stopwords
- Remove any html tags or urls




In [39]:
## 1. Lower All the cases
df['reviewText']=df['reviewText'].str.lower()

In [40]:
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [41]:
from bs4 import BeautifulSoup
## Removing special characters
df['reviewText']=df['reviewText'].apply(lambda x:re.sub('[^a-z A-z 0-9-]+', '',x))
## Remove the stopswords
df['reviewText']=df['reviewText'].apply(lambda x:" ".join([y for y in x.split() if y not in stopwords.words('english')]))
## Remove url
df['reviewText']=df['reviewText'].apply(lambda x: re.sub(r'(http|https|ftp|ssh)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?', '' , str(x)))
## Remove html tags
df['reviewText']=df['reviewText'].apply(lambda x: BeautifulSoup(x, 'lxml').get_text())
## Remove any additional spaces
df['reviewText']=df['reviewText'].apply(lambda x: " ".join(x.split()))

In [19]:
df.head()

,overall,reviewText
0,0,issues
1,1,purchased device worked advertised never much ...
2,0,works expected sprung higher capacity think ma...
3,1,think worked greathad diff bran 64gb card went...
4,1,bought retail packaging arrived legit orange e...


Lemmatization with pos tagging, to ensure best input possible


In [42]:
from nltk.stem import WordNetLemmatizer
lemma=WordNetLemmatizer()

In [43]:
from nltk.corpus import wordnet
from nltk import pos_tag, word_tokenize

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN  # Default to noun


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [44]:
lemma=WordNetLemmatizer()
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger_eng')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


True

In [45]:
def lemmatize_sentence(sentence):
    words = word_tokenize(sentence)
    tagged_words = pos_tag(words)
    lemmatized = [lemma.lemmatize(word, get_wordnet_pos(tag)) for word, tag in tagged_words]
    return ' '.join(lemmatized)
df['reviewText'] = df['reviewText'].apply(lemmatize_sentence)

In [24]:
df.head()

,overall,reviewText
0,0,issue
1,1,purchase device work advertised never much pho...
2,0,work expect sprung high capacity think make bi...
3,1,think work greathad diff bran 64gb card go sou...
4,1,buy retail packaging arrive legit orange envel...


Seperating the data into training set and testing set


Training the model using Word2Vec

In [26]:
%pip install gensim

In [27]:
%pip install --upgrade numpy gensim --quiet


In [28]:
import gensim
from gensim.models import Word2Vec, KeyedVectors

In [29]:
import gensim.downloader as api
wv = api.load('word2vec-google-news-300')

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [46]:
import numpy as np
def sentence_vector(tokens):
    word_vecs = [model.wv[word] for word in tokens if word in model.wv]
    if len(word_vecs) == 0:
        return np.zeros(model.vector_size)
    return np.mean(word_vecs, axis=0)


In [49]:
# 1. Preprocess and tokenize all text
df['tokens'] = df['reviewText'].apply(lambda x: x.split())

# 2. Train Word2Vec on all tokens
model = Word2Vec(sentences=df['tokens'].tolist())

# 3. Convert each sentence to vector using the trained model
df['vector'] = df['tokens'].apply(sentence_vector)

# 4. Now split into train and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df['vector'].tolist(), df['overall'], test_size=0.2)


In [53]:
from sklearn.ensemble import RandomForestClassifier
classifier=RandomForestClassifier()

In [54]:
classifier.fit(X_train,y_train)

RandomForestClassifier()

In [55]:
y_pred=classifier.predict(X_test)

In [56]:
from sklearn.metrics import accuracy_score,classification_report
print(accuracy_score(y_test,y_pred))

0.832146490335707


In [57]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.57      0.29      0.38       178
           1       0.86      0.95      0.90       805

    accuracy                           0.83       983
   macro avg       0.72      0.62      0.64       983
weighted avg       0.81      0.83      0.81       983

